In [4]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import math
import sys
import json

# On affiche les resultat de la fonctions
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(
            manager.IndexToNode(index), route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main(argv):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
#     On recupere le Dataset
    with open("Dataset/Data.json".format(argv[0]), 'r') as json_file:
        data = json.load(json_file)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
#     print (assignment)
    if assignment:
        print('Nbr de vehicules {}'.format(data['num_vehicles']))
        print('Capacité des vehicules {}'.format(data['vehicle_capacities']))
        print_solution(data, manager, routing, assignment)
        


if __name__ == '__main__':
    main(sys.argv[1:])

Assignment(Capacity0 (0) | Capacity1 (43) | Capacity2 (7) | Capacity3 (41) | Capacity4 (27) | Capacity5 (8) | Capacity6 (32) | Capacity7 (53) | Capacity8 (30) | Capacity9 (7) | Capacity10 (7) | Capacity11 (16) | Capacity12 (35) | Capacity13 (8) | Capacity14 (89) | Capacity15 (7) | Capacity16 (7) | Capacity17 (63) | Capacity18 (10) | Capacity19 (11) | Capacity20 (53) | Capacity21 (67) | Capacity22 (33) | Capacity23 (50) | Capacity24 (91) | Capacity25 (95) | Capacity26 (53) | Capacity27 (39) | Capacity28 (24) | Capacity29 (32) | Capacity30 (7) | Capacity31 (38) | Capacity32 (37) | Capacity33 (73) | Capacity34 (56) | Capacity35 (37) | Capacity36 (81) | Capacity37 (27) | Capacity38 (38) | Capacity39 (82) | Capacity40 (39) | Capacity41 (26) | Capacity42 (51) | Capacity43 (7) | Capacity44 (73) | Capacity45 (15) | Capacity46 (25) | Capacity47 (26) | Capacity48 (66) | Capacity49 (7) | Capacity50 (0) | Capacity51 (0) | Capacity52 (0) | Capacity53 (0) | Capacity54 (0) | Capacity55 (0) | Capacity